In [1]:
from google.colab import files
uploaded = files.upload()

Saving anime.csv to anime.csv


In [4]:
import pandas as pd
df = pd.read_csv("/content/anime.csv")

print("Jumlah data awal:", len(df))
print("\nCek missing value:")
print(df.isnull().sum())

df['episodes'] = pd.to_numeric(df['episodes'], errors='coerce')

for col in df.columns:
   if df[col].dtype in ['float64', 'int64']:
    df[col] = df[col].fillna(df[col].median())
else:
    df[col] = df[col].fillna(df[col].mode()[0])

df.drop_duplicates(inplace=True)

numeric_cols = ['episodes', 'rating', 'members']
Q1 = df[numeric_cols].quantile(0.25)
Q3 = df[numeric_cols].quantile(0.75)
IQR = Q3 - Q1

df = df[~((df[numeric_cols] < (Q1 - 1.5 * IQR)) | (df[numeric_cols] > (Q3 + 1.5 * IQR))).any(axis=1)]

print("\nJumlah data setelah cleaning:", len(df))
print("\nCek missing value setelah cleaning:")
print(df.isnull().sum())


Jumlah data awal: 12294

Cek missing value:
anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

Jumlah data setelah cleaning: 9164

Cek missing value setelah cleaning:
anime_id     0
name         0
genre       58
type        24
episodes     0
rating       0
members      0
dtype: int64


In [10]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
numeric_cols = ['episodes', 'rating', 'members']


scaler_minmax = MinMaxScaler()
df_normalized = df.copy()
df_normalized[numeric_cols] = scaler_minmax.fit_transform(df[numeric_cols])

scaler_standard = StandardScaler()
df_standardized = df.copy()
df_standardized[numeric_cols] = scaler_standard.fit_transform(df[numeric_cols])

print("\nContoh hasil normalisasi:")
print(df_normalized[numeric_cols].head())

print("\nContoh hasil standarisasi:")
print(df_standardized[numeric_cols].head())


Contoh hasil normalisasi:
     episodes  rating   members
41   0.037037   0.938  0.729787
63   0.037037   0.920  0.478096
103  0.037037   0.900  0.577213
140  0.185185   0.884  0.664010
165  0.000000   0.876  0.938747

Contoh hasil standarisasi:
     episodes    rating   members
41  -0.367294  2.762450  2.710532
63  -0.367294  2.658552  1.550668
103 -0.367294  2.543109  2.007428
140  0.228675  2.450755  2.407415
165 -0.516286  2.404578  3.673479


In [17]:
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer

df['genre'] = df['genre'].astype(str)

le = LabelEncoder()
df['type_encoded'] = le.fit_transform(df['type'])

df['genre_list'] = df['genre'].apply(lambda x: [g.strip() for g in x.split(',') if g.strip().lower() != 'nan' and g.strip() != ''])

mlb = MultiLabelBinarizer()
genre_encoded = pd.DataFrame(mlb.fit_transform(df['genre_list']), columns=mlb.classes_)


df_encoded = pd.concat([df, genre_encoded], axis=1)
if 'nan' in df_encoded.columns:
    df_encoded.drop(columns=['nan'], inplace=True)
print("\nContoh hasil data setelah encoding:")
print(df_encoded.head(3))


Contoh hasil data setelah encoding:
   anime_id                                         name  \
0     32366                   Gintama°: Aizome Kaori-hen   
1     21899  Gintama: Yorinuki Gintama-san on Theater 2D   
2     11917                          Major: World Series   

                                               genre   type  episodes  rating  \
0                                     Comedy, Parody    OVA       2.0    8.69   
1  Action, Comedy, Historical, Parody, Samurai, S...  Movie       2.0    8.60   
2                              Comedy, Drama, Sports    OVA       2.0    8.50   

   members  type_encoded                                         genre_list  \
0    16947             3                                   [Comedy, Parody]   
1    11104             0  [Action, Comedy, Historical, Parody, Samurai, ...   
2    13405             3                            [Comedy, Drama, Sports]   

   Action  ...  Shounen Ai  Slice of Life  Space  Sports  Super Power  \
0      

In [21]:
genre_cols = [col for col in df.columns if col not in ['anime_id', 'name', 'genre', 'type', 'episodes', 'rating', 'members', 'type_encoded', 'genre_list']]
df['jumlah_genre'] = df[genre_cols].sum(axis=1)
df['popularitas'] = df['rating'] * df['members']
print("Fitur baru ditambahkan: jumlah_genre & popularitas")
print("\nContoh hasil data setelah feature engineering:")
print(df[['name', 'rating', 'members', 'jumlah_genre', 'popularitas']].head(5))

Fitur baru ditambahkan: jumlah_genre & popularitas

Contoh hasil data setelah feature engineering:
                                          name  rating  members  jumlah_genre  \
0                   Gintama°: Aizome Kaori-hen    8.69    16947     441808.29   
1  Gintama: Yorinuki Gintama-san on Theater 2D    8.60    11104     286483.20   
2                          Major: World Series    8.50    13405     341827.50   
3               Mobile Suit Gundam: The Origin    8.42    15420     389509.20   
4   Detective Conan Movie 13: The Raven Chaser    8.38    21798     548001.72   

   popularitas  
0    147269.43  
1     95494.40  
2    113942.50  
3    129836.40  
4    182667.24  


In [24]:
from sklearn.model_selection import train_test_split
X = df.drop(columns=['rating', 'name', 'genre', 'genre_list'])
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    shuffle=True
)
print("Ukuran data training :", X_train.shape)
print("Ukuran data testing  :", X_test.shape)

Ukuran data training : (7331, 7)
Ukuran data testing  : (1833, 7)


<H1>Data Preprocessing</H1>
Tahap splitting data merupakan proses penting dalam machine learning untuk memisahkan dataset menjadi dua bagian utama, yaitu data training dan data testing. Data training digunakan untuk melatih model agar mampu mengenali pola dari data, sedangkan data testing digunakan untuk menguji kemampuan model terhadap data yang belum pernah dilihat sebelumnya. Tujuan utama dari pembagian ini adalah agar model tidak hanya menghafal data (overfitting), tetapi juga mampu melakukan generalisasi dengan baik terhadap data baru.

Dalam penelitian ini, pembagian data dilakukan dengan menggunakan fungsi `train_test_split` dari library `scikit-learn`, dengan proporsi **80% untuk data training dan 20% untuk data testing**. Pembagian ini merupakan standar umum yang memberikan keseimbangan antara jumlah data untuk pelatihan dan pengujian. Parameter `random_state=42` digunakan agar hasil pembagian selalu konsisten setiap kali kode dijalankan, sehingga eksperimen dapat direproduksi ulang. Selain itu, parameter `shuffle=True` diterapkan agar data diacak terlebih dahulu sebelum dibagi, sehingga urutan data (misalnya berdasarkan tahun rilis atau genre anime) tidak mempengaruhi hasil pembagian dan tidak menimbulkan bias pada model.

Secara keseluruhan, proses *splitting data* ini sangat penting untuk memastikan bahwa hasil evaluasi model bersifat objektif. Dengan cara ini, performa model dapat diukur secara adil menggunakan data yang benar-benar baru bagi model, sehingga tingkat akurasi yang diperoleh lebih mencerminkan kemampuan model dalam melakukan prediksi di dunia nyata.
